#BAG OF WORDS

NOMBRE:Alexander Javier Huaman Guevara

CODIGO:150394

In [1]:
!pip install pyspark


     |████████████████████████████████| 281.3 MB 32 kB/s 
     |████████████████████████████████| 198 kB 45.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=310ba736fa227af6ada73733d009a5177314e229fee406da67b1f41b2598e298
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


BAG OF WORDS

In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("appName")
sc = SparkContext(conf=conf)

In [3]:
def bow(text):
    x=text.strip().lower().split()
    return list(filter(lambda x:len(x)>2,x))

In [4]:
Corpus=["This is a document"
"A book by Mary",
        "Newspaper article that  I wrote a week ago"]
Texto=sc.parallelize(Corpus,6)
Texto=Texto.map(bow)
Texto.take(2)

[['this', 'documenta', 'book', 'mary'],
 ['newspaper', 'article', 'that', 'wrote', 'week', 'ago']]

TF

In [7]:
import math
def tf(text):
    total=len(text)
    DocS=list(set(text))
    maping=list(map(lambda x:(x,text.count(x)),DocS))
    tf=list(map(lambda x:(x[0],1+math.log10(x[1]/total)),maping))
    return (tf)

In [8]:
Corpus=["This is a document"
"A book by Mary",
        "Newspaper article that  I wrote a week ago"]
Texto=sc.parallelize(Corpus,4)
Texto=Texto.map(bow)
TextoTF=Texto.map(tf)
TextoTF.take(2)

[[('book', 0.3979400086720376),
  ('this', 0.3979400086720376),
  ('documenta', 0.3979400086720376),
  ('mary', 0.3979400086720376)],
 [('article', 0.22184874961635637),
  ('newspaper', 0.22184874961635637),
  ('that', 0.22184874961635637),
  ('wrote', 0.22184874961635637),
  ('week', 0.22184874961635637),
  ('ago', 0.22184874961635637)]]

IDF

In [10]:
import math
def idf(text,corpus):
    DocP=text.flatMap(lambda x:x)
    DocP=DocP.map(lambda x:x[0]).distinct()   
    total=corpus.count()
    corpus2=corpus.collect()
    idf=DocP.map(lambda x:list(map(lambda y:(x,1 if x in y else 0),corpus2)))
    idf2=idf.flatMap(lambda x:x)
    idf2=idf2.groupByKey().map(lambda x:(x[0],sum(x[1])))
    idf2=idf2.map(lambda x:(x[0],math.log10(1+total/x[1])))
    return idf2

In [11]:
Corpus=["This is a document"
"A book by Mary",
        "Newspaper article that  I wrote a week ago"]
Texto=sc.parallelize(Corpus,4)
Texto=Texto.map(bow)
Textotf=Texto.map(TF)
Textoidf=idf(Textotf,Texto)
Textoidf.collect()

[('book', 0.47712125471966244),
 ('documenta', 0.47712125471966244),
 ('mary', 0.47712125471966244),
 ('article', 0.47712125471966244),
 ('that', 0.47712125471966244),
 ('this', 0.47712125471966244),
 ('newspaper', 0.47712125471966244),
 ('week', 0.47712125471966244),
 ('ago', 0.47712125471966244),
 ('wrote', 0.47712125471966244)]

TFIDF

In [12]:
import math
def RecIDF(word,corpusIDF):
        L=corpusIDF
        EL=list(filter(lambda x:x[0]==word,L))
        return EL[0][1]


def TFIDF(TextoTF,TextoIDF):
    """Funcion que calcula el IDF para cada Doc de TF
    Args:
        Doc [[(str,double)]]: texto TF
        Doc [(str,double)]:texto IDF

    Returns:
        [(str,double)]: texto con el TFIDF de cada palabra en DOC"""
    corpusIDF=TextoIDF.collect()
    TFIDF=TextoTF.map(lambda x:list(map((lambda y:(y[0],y[1]*RecIDF(y[0],corpusIDF))),x)))
    return  TFIDF


In [13]:
Corpus=["This is a document"
"A book by Mary",
        "Newspaper article that  I wrote a week ago"]
Texto=sc.parallelize(Corpus,4)
Texto=Texto.map(bow)
TextoTF=Texto.map(TF)
TextoIDF=idf(TextoTF,Texto)
TextoTFIDF=TFIDF(TextoTF,TextoIDF)
TextoTFIDF.collect()

[[('book', 0.18986563624075592),
  ('this', 0.18986563624075592),
  ('documenta', 0.18986563624075592),
  ('mary', 0.18986563624075592)],
 [('article', 0.10584875377494418),
  ('newspaper', 0.10584875377494418),
  ('that', 0.10584875377494418),
  ('wrote', 0.10584875377494418),
  ('week', 0.10584875377494418),
  ('ago', 0.10584875377494418)]]

nGrams

In [14]:
def taken(n,tokens):
 
    if len(tokens)>=n:
        tokens=list(map(lambda x:str(x),tokens))
        val=((" ").join(tokens[0:n]))
    else:
        val=""
    return val
def tails(tokens):
    tailsss=[tokens]
    for i in range(len(tokens)-1):
        tailsss.append(tailsss[-1][1:])
    return tailsss
def nGrams(n,tokens):
    tokens=tokens.split()
    tokens=tails(tokens)
    Tok=list(map(lambda x:taken(n,x),tokens))
    #elimino los vacios
    Tok=list(filter(lambda x:len(x)>0,Tok))
    return Tok

In [15]:
Corpus=["This is a document"
"A book by Mary",
        "Newspaper article that  I wrote a week ago"]
Corpus=sc.parallelize(Corpus,4)

Bigrama=Corpus.map(lambda x:nGrams(2,x))
print('Bigrama',Bigrama.take(5))
print()

Trigramas=Corpus.map(lambda x:nGrams(3,x))
print("Trigramas",Trigramas.take(5))
print()

Tetragramas=Corpus.map(lambda x:nGrams(4,x))
print("Tetragramas",Tetragramas.take(5))
print()

Bigrama [['This is', 'is a', 'a documentA', 'documentA book', 'book by', 'by Mary'], ['Newspaper article', 'article that', 'that I', 'I wrote', 'wrote a', 'a week', 'week ago']]

Trigramas [['This is a', 'is a documentA', 'a documentA book', 'documentA book by', 'book by Mary'], ['Newspaper article that', 'article that I', 'that I wrote', 'I wrote a', 'wrote a week', 'a week ago']]

Tetragramas [['This is a documentA', 'is a documentA book', 'a documentA book by', 'documentA book by Mary'], ['Newspaper article that I', 'article that I wrote', 'that I wrote a', 'I wrote a week', 'wrote a week ago']]

